In [ ]:
#Importing Libraries
import os
import cv2
import shutil
import numpy as np
from os import path
from tqdm import tqdm
from google.colab import files
from IPython.display import clear_output
from moviepy.editor import VideoFileClip, AudioFileClip, concatenate_videoclips, clips_array

In [ ]:
#Mounting Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#Clonin Wav2lip from github and setting up folders along with installing the requirements

!git clone https://github.com/indianajson/wav2lip-HD.git
basePath = "/content/wav2lip-HD"
%cd {basePath}

wav2lipFolderName = 'Wav2Lip-master'
gfpganFolderName = 'GFPGAN-master'
wav2lipPath = basePath + '/' + wav2lipFolderName
gfpganPath = basePath + '/' + gfpganFolderName

!wget 'https://www.adrianbulat.com/downloads/python-fan/s3fd-619a316812.pth' -O {wav2lipPath}'/face_detection/detection/sfd/s3fd.pth'
!wget 'https://iiitaphyd-my.sharepoint.com/personal/radrabha_m_research_iiit_ac_in/_layouts/15/download.aspx?share=EdjI7bZlgApMqsVoEUUXpLsBxqXbn5z8VTmoxp55YNDcIA' -O {wav2lipPath}'/checkpoints/wav2lip_gan.pth'
!gdown https://drive.google.com/uc?id=1fQtBSYEyuai9MjBOF8j7zZ4oQ9W2N64q --output {wav2lipPath}'/checkpoints/'

!pip install -r requirements.txt
!pip install -U librosa==0.8.1
!pip install basicsr facexlib
!mkdir inputs

!cd $gfpganFolderName && python setup.py develop
!wget https://github.com/TencentARC/GFPGAN/releases/download/v1.3.4/GFPGANv1.4.pth -P {gfpganFolderName}'/experiments/pretrained_models'

%cd {basePath}

clear_output()

In [ ]:
#Copying sample files from drive to wav2lip-HD/inputs folder
file_pairs = [
    ('/content/drive/MyDrive/task/org_vid.mp4', '/content/wav2lip-HD/inputs/org_vid.mp4'),
    ('/content/drive/MyDrive/task/len_audio.wav', '/content/wav2lip-HD/inputs/len_audio.wav')
]

for source_path, destination_path in file_pairs:
    shutil.copyfile(source_path, destination_path)

'/content/wav2lip-HD/inputs/len_audio.wav'

In [ ]:
#Upload inference_gfpgan.py file to /content/
paths=['/content/wav2lip-HD/colab.ipynb','/content/wav2lip-HD/GFPGAN-master/inference_gfpgan.py']

for file_path in paths:
  try:
      os.remove(file_path)
      print(f"File '{file_path}' has been deleted.")
  except FileNotFoundError:
      print(f"File '{file_path}' not found.")
  except Exception as e:
      print(f"An error occurred while trying to delete '{file_path}': {e}")


source_file_paths = ['/content/colab.ipynb', '/content/inference_gfpgan.py']
destination_folders = ['/content/wav2lip-HD','/content/wav2lip-HD/GFPGAN-master']

for (source_file_path, destination_folder) in zip(source_file_paths, destination_folders):
  try:
      destination_file_path = os.path.join(destination_folder, os.path.basename(source_file_path))
      os.rename(source_file_path, destination_file_path)
      print(f"File '{source_file_path}' has been moved to '{destination_file_path}'.")
  except FileNotFoundError:
      print(f"File '{source_file_path}' not found.")
  except Exception as e:
      print(f"An error occurred while trying to move '{source_file_path}': {e}")

File '/content/wav2lip-HD/colab.ipynb' has been deleted.
File '/content/wav2lip-HD/GFPGAN-master/inference_gfpgan.py' has been deleted.
File '/content/colab.ipynb' not found.
File '/content/inference_gfpgan.py' has been moved to '/content/wav2lip-HD/GFPGAN-master/inference_gfpgan.py'.


### IMP: Updating inference file in Wav2lip folder to skip frames with no face while face detection

In [ ]:
# Define the file path
file_path = '/content/wav2lip-HD/inference_gfpgan.py'

text_to_write = '''from os import listdir, path
import numpy as np
import scipy, cv2, os, sys, argparse, audio
import json, subprocess, random, string
from tqdm import tqdm
from glob import glob
import torch, face_detection
from models import Wav2Lip
import platform

parser = argparse.ArgumentParser(description='Inference code to lip-sync videos in the wild using Wav2Lip models')

parser.add_argument('--checkpoint_path', type=str,
					help='Name of saved checkpoint to load weights from', required=True)

parser.add_argument('--face', type=str,
					help='Filepath of video/image that contains faces to use', required=True)
parser.add_argument('--audio', type=str,
					help='Filepath of video/audio file to use as raw audio source', required=True)
parser.add_argument('--outfile', type=str, help='Video path to save result. See default for an e.g.',
								default='results/result_voice.mp4')

parser.add_argument('--static', type=bool,
					help='If True, then use only first video frame for inference', default=False)
parser.add_argument('--fps', type=float, help='Can be specified only if input is a static image (default: 25)',
					default=25., required=False)

parser.add_argument('--pads', nargs='+', type=int, default=[0, 10, 0, 0],
					help='Padding (top, bottom, left, right). Please adjust to include chin at least')

parser.add_argument('--face_det_batch_size', type=int,
					help='Batch size for face detection', default=16)
parser.add_argument('--wav2lip_batch_size', type=int, help='Batch size for Wav2Lip model(s)', default=128)

parser.add_argument('--resize_factor', default=1, type=int,
			help='Reduce the resolution by this factor. Sometimes, best results are obtained at 480p or 720p')

parser.add_argument('--crop', nargs='+', type=int, default=[0, -1, 0, -1],
					help='Crop video to a smaller region (top, bottom, left, right). Applied after resize_factor and rotate arg. '
					'Useful if multiple face present. -1 implies the value will be auto-inferred based on height, width')

parser.add_argument('--box', nargs='+', type=int, default=[-1, -1, -1, -1],
					help='Specify a constant bounding box for the face. Use only as a last resort if the face is not detected.'
					'Also, might work only if the face is not moving around much. Syntax: (top, bottom, left, right).')

parser.add_argument('--rotate', default=False, action='store_true',
					help='Sometimes videos taken from a phone can be flipped 90deg. If true, will flip video right by 90deg.'
					'Use if you get a flipped result, despite feeding a normal looking video')

parser.add_argument('--nosmooth', default=False, action='store_true',
					help='Prevent smoothing face detections over a short temporal window')

args = parser.parse_args()
args.img_size = 96

if os.path.isfile(args.face) and args.face.split('.')[1] in ['jpg', 'png', 'jpeg']:
	args.static = True

def get_smoothened_boxes(boxes, T):
	for i in range(len(boxes)):
		if i + T > len(boxes):
			window = boxes[len(boxes) - T:]
		else:
			window = boxes[i : i + T]
		boxes[i] = np.mean(window, axis=0)
	return boxes

def face_detect(images):
    detector = face_detection.FaceAlignment(face_detection.LandmarksType._2D, flip_input=False, device=device)
    batch_size = args.face_det_batch_size

    while 1:
        predictions = []
        try:
            for i in tqdm(range(0, len(images), batch_size)):
                predictions.extend(detector.get_detections_for_batch(np.array(images[i:i + batch_size])))
        except RuntimeError:
            if batch_size == 1:
                raise RuntimeError('Image too big to run face detection on GPU. Please use the --resize_factor argument')
            batch_size //= 2
            print('Recovering from OOM error; New batch size: {}'.format(batch_size))
            continue
        break

    results = []
    pady1, pady2, padx1, padx2 = args.pads
    for rect, image in zip(predictions, images):
        if rect is None:
            # Add the original frame without face to results
            results.append([image, []])
        else:
            y1 = max(0, rect[1] - pady1)
            y2 = min(image.shape[0], rect[3] + pady2)
            x1 = max(0, rect[0] - padx1)
            x2 = min(image.shape[1], rect[2] + padx2)

            results.append([image[y1:y2, x1:x2], (y1, y2, x1, x2)])

    return results


def datagen(frames, mels):
    img_batch, mel_batch, frame_batch, coords_batch = [], [], [], []

    if args.box[0] == -1:
        if not args.static:
            face_det_results = face_detect(frames) # BGR2RGB for CNN face detection
        else:
            face_det_results = face_detect([frames[0]])
    else:
        print('Using the specified bounding box instead of face detection...')
        y1, y2, x1, x2 = args.box
        face_det_results = [[f[y1: y2, x1:x2], (y1, y2, x1, x2)] for f in frames]

    for i, m in enumerate(mels):
        idx = 0 if args.static else i%len(frames)
        frame_to_save = frames[idx].copy()
        face, coords = face_det_results[idx].copy()

        if len(coords) == 0: # Check if there are no face coordinates
            # Add a random matrix of shape (96x96x3) to img_batch and an empty list to coords_batch
            face = np.random.rand(args.img_size, args.img_size, 3) * 255
            coords_batch.append([])
        else:
            face = cv2.resize(face, (args.img_size, args.img_size))
            coords_batch.append(coords)

        img_batch.append(face)
        mel_batch.append(m)
        frame_batch.append(frame_to_save)

        if len(img_batch) >= args.wav2lip_batch_size:
            img_batch, mel_batch = np.asarray(img_batch), np.asarray(mel_batch)

            img_masked = img_batch.copy()
            img_masked[:, args.img_size//2:] = 0

            img_batch = np.concatenate((img_masked, img_batch), axis=3) / 255.
            mel_batch = np.reshape(mel_batch, [len(mel_batch), mel_batch.shape[1], mel_batch.shape[2], 1])

            yield img_batch, mel_batch, frame_batch, coords_batch
            img_batch, mel_batch, frame_batch, coords_batch = [], [], [], []

    if len(img_batch) > 0:
        img_batch, mel_batch = np.asarray(img_batch), np.asarray(mel_batch)

        img_masked = img_batch.copy()
        img_masked[:, args.img_size//2:] = 0

        img_batch = np.concatenate((img_masked, img_batch), axis=3) / 255.
        mel_batch = np.reshape(mel_batch, [len(mel_batch), mel_batch.shape[1], mel_batch.shape[2], 1])

        yield img_batch, mel_batch, frame_batch, coords_batch

mel_step_size = 16
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using {} for inference.'.format(device))

def _load(checkpoint_path):
	if device == 'cuda':
		checkpoint = torch.load(checkpoint_path)
	else:
		checkpoint = torch.load(checkpoint_path,
								map_location=lambda storage, loc: storage)
	return checkpoint

def load_model(path):
	model = Wav2Lip()
	print("Load checkpoint from: {}".format(path))
	checkpoint = _load(path)
	s = checkpoint["state_dict"]
	new_s = {}
	for k, v in s.items():
		new_s[k.replace('module.', '')] = v
	model.load_state_dict(new_s)

	model = model.to(device)
	return model.eval()

def main():
    if not os.path.isfile(args.face):
        raise ValueError('--face argument must be a valid path to video/image file')

    elif args.face.split('.')[1] in ['jpg', 'png', 'jpeg']:
        full_frames = [cv2.imread(args.face)]
        fps = args.fps

    else:
        video_stream = cv2.VideoCapture(args.face)
        fps = video_stream.get(cv2.CAP_PROP_FPS)

        print('Reading video frames...')

        full_frames = []
        while 1:
            still_reading, frame = video_stream.read()
            if not still_reading:
                video_stream.release()
                break
            if args.resize_factor > 1:
                frame = cv2.resize(frame, (frame.shape[1]//args.resize_factor, frame.shape[0]//args.resize_factor))

            if args.rotate:
                frame = cv2.rotate(frame, cv2.cv2.ROTATE_90_CLOCKWISE)

            y1, y2, x1, x2 = args.crop
            if x2 == -1: x2 = frame.shape[1]
            if y2 == -1: y2 = frame.shape[0]

            frame = frame[y1:y2, x1:x2]

            full_frames.append(frame)

    print("Number of frames available for inference: " + str(len(full_frames)))

    if not args.audio.endswith('.wav'):
        print('Extracting raw audio...')
        command = 'ffmpeg -y -i {} -strict -2 {}'.format(args.audio, 'temp/temp.wav')

        subprocess.call(command, shell=True)
        args.audio = 'temp/temp.wav'

    wav = audio.load_wav(args.audio, 16000)
    mel = audio.melspectrogram(wav)
    print(mel.shape)

    if np.isnan(mel.reshape(-1)).sum() > 0:
        raise ValueError('Mel contains nan! Using a TTS voice? Add a small epsilon noise to the wav file and try again')

    mel_chunks = []
    mel_idx_multiplier = 80. / fps
    i = 0
    while 1:
        start_idx = int(i * mel_idx_multiplier)
        if start_idx + mel_step_size > len(mel[0]):
            mel_chunks.append(mel[:, len(mel[0]) - mel_step_size:])
            break
        mel_chunks.append(mel[:, start_idx: start_idx + mel_step_size])
        i += 1

    print("Length of mel chunks: {}".format(len(mel_chunks)))

    full_frames = full_frames[:len(mel_chunks)]

    batch_size = args.wav2lip_batch_size
    gen = datagen(full_frames.copy(), mel_chunks)

    for i, (img_batch, mel_batch, frames, coords) in enumerate(tqdm(gen, total=int(np.ceil(float(len(mel_chunks))/batch_size)))):
        if i == 0:
            model = load_model(args.checkpoint_path)
            print("Model loaded")

            frame_h, frame_w = full_frames[0].shape[:-1]
            out = cv2.VideoWriter('temp/result.avi',
                                  cv2.VideoWriter_fourcc(*'DIVX'), fps, (frame_w, frame_h))

        img_batch = torch.FloatTensor(np.transpose(img_batch, (0, 3, 1, 2))).to(device)
        mel_batch = torch.FloatTensor(np.transpose(mel_batch, (0, 3, 1, 2))).to(device)

        with torch.no_grad():
            pred = model(mel_batch, img_batch)

        pred = pred.cpu().numpy().transpose(0, 2, 3, 1) * 255.

        for p, f, c in zip(pred, frames, coords):

            # y1, y2, x1, x2 = c  <- Move this line to lie 272

            if len(c) == 0:  # Check if there are no face coordinates (no face detected)
                out.write(f)  # Write the original frame without lip-syncing
            else:
                y1, y2, x1, x2 = c # <- here
                p = cv2.resize(p.astype(np.uint8), (x2 - x1, y2 - y1))
                f[y1:y2, x1:x2] = p
                out.write(f)  # Write the lip-synced frame to the output video

    out.release()

    command = 'ffmpeg -y -i {} -i {} -strict -2 -q:v 1 {}'.format(args.audio, 'temp/result.avi', args.outfile)
    subprocess.call(command, shell=platform.system() != 'Windows')


if __name__ == '__main__':
    main()'''

with open(file_path, 'w') as file:
    file.write(text_to_write)

### Getting output from Wav2Lip

In [ ]:
outputPath = basePath+'/outputs'
# inputAudio = '/content/drive/MyDrive/taks/Sample_input_audio.wav' #@param{type:"string"}
inputAudioPath = "/content/wav2lip-HD/inputs/len_audio.wav"#basePath + '/inputs/' + inputAudio
# inputVideo = '/content/drive/MyDrive/taks/Sample_input_vid.mp4' #@param{type:"string"}
inputVideoPath = "/content/wav2lip-HD/inputs/org_vid.mp4"#basePath + '/inputs/'+inputVideo
lipSyncedOutputPath = basePath + '/outputs/result.mp4'
model = "wav2lip" #@param ["wav2lip", "wav2lip_gan"] {type:"string"}

if not os.path.exists(outputPath):
  os.makedirs(outputPath)

!cd $wav2lipFolderName && python inference.py \
--checkpoint_path checkpoints/{model}.pth \
--face {inputVideoPath} \
--audio {inputAudioPath} \
--outfile {lipSyncedOutputPath} \
--nosmooth

# clear_output()
print("Video is ready in the output folder.")

Using cuda for inference.
Reading video frames...
Number of frames available for inference: 1720
(80, 6814)
Length of mel chunks: 1701
  0% 0/14 [00:00<?, ?it/s]
  0% 0/107 [00:00<?, ?it/s]
  1% 1/107 [00:39<1:09:29, 39.34s/it]
  2% 2/107 [00:42<49:45, 28.43s/it]  
  3% 3/107 [00:44<35:46, 20.64s/it]
  4% 4/107 [00:47<26:01, 15.16s/it]
  5% 5/107 [00:49<19:11, 11.29s/it]
  6% 6/107 [00:51<14:29,  8.61s/it]
  7% 7/107 [00:54<11:18,  6.78s/it]
  7% 8/107 [00:56<08:57,  5.43s/it]
  8% 9/107 [00:58<07:19,  4.48s/it]
  9% 10/107 [01:01<06:10,  3.82s/it]
 10% 11/107 [01:03<05:22,  3.36s/it]
 11% 12/107 [01:05<04:50,  3.06s/it]
 12% 13/107 [01:08<04:39,  2.97s/it]
 13% 14/107 [01:11<04:25,  2.85s/it]
 14% 15/107 [01:13<04:12,  2.74s/it]
 15% 16/107 [01:16<04:05,  2.70s/it]
 16% 17/107 [01:18<04:04,  2.71s/it]
 17% 18/107 [01:21<04:08,  2.79s/it]
 18% 19/107 [01:24<04:04,  2.78s/it]
 19% 20/107 [01:27<03:53,  2.69s/it]
 20% 21/107 [01:29<03:42,  2.59s/it]
 21% 22/107 [01:31<03:32,  2.50s/it]
 

### Using another GAN for quality boost

In [ ]:
inputVideoPath = outputPath+'/result.mp4'
unProcessedFramesFolderPath = outputPath+'/frames'

if not os.path.exists(unProcessedFramesFolderPath):
  os.makedirs(unProcessedFramesFolderPath)

vidcap = cv2.VideoCapture(inputVideoPath)
numberOfFrames = int(vidcap.get(cv2.CAP_PROP_FRAME_COUNT))
fps = vidcap.get(cv2.CAP_PROP_FPS)
print("FPS: ", fps, "Frames: ", numberOfFrames)

for frameNumber in tqdm(range(numberOfFrames)):
    _,image = vidcap.read()
    cv2.imwrite(path.join(unProcessedFramesFolderPath, str(frameNumber).zfill(4)+'.jpg'), image)

FPS:  20.0 Frames:  1701


100%|██████████| 1701/1701 [00:30<00:00, 55.37it/s]


In [ ]:
!cd $gfpganFolderName && python inference_gfpgan.py -i $unProcessedFramesFolderPath -o $outputPath -v 1.4 -s 2 --only_center_face --bg_upsampler None

restoredFramesPath = '/content/drive/MyDrive/task/outt' + '/restored_imgs/'
processedVideoOutputPath = outputPath

dir_list = os.listdir(restoredFramesPath)
dir_list.sort()



#Get FPS of original video for writer
inputVideoPath = outputPath+'/result.mp4'
vidcap = cv2.VideoCapture(inputVideoPath)
fps = vidcap.get(cv2.CAP_PROP_FPS)

batch = 0
batchSize = 900
for i in tqdm(range(0, len(dir_list), batchSize)):
  img_array = []
  start, end = i, i+batchSize
  print("processing ", start, end, end="\r")
  for filename in  tqdm(dir_list[start:end]):
      filename = restoredFramesPath+filename;
      img = cv2.imread(filename)
      if img is None:
        continue
      height, width, layers = img.shape
      size = (width,height)
      img_array.append(img)
  out = cv2.VideoWriter(processedVideoOutputPath+'/output_'+str(batch).zfill(4)+'.mp4',cv2.VideoWriter_fourcc(*'DIVX'), fps, size)
  batch = batch + 1

  for i in range(len(img_array)):
    out.write(img_array[i])
  out.release()

# clear_output()

print("Video upscaling complete.")

  0%|          | 0/2 [00:00<?, ?it/s]


 50%|█████     | 1/2 [00:41<00:41, 41.92s/it]


100%|██████████| 2/2 [01:15<00:00, 37.95s/it]

Video upscaling complete.


### Adding audio to GAN output

In [ ]:
%cd "/content/drive/MyDrive/task/output/"

#Processing in batchSize of 900 frames with a total of 1701 frames implies approximately 3 videos.

#NOTE: YOU CAN PROCESS ALL THE FRAMES IN A SINGLE GO IF YOU HAVE SUFFICIENT RAM ;)

video1 = VideoFileClip("output_0000.mp4")
video2 = VideoFileClip("output_0001.mp4")
video3 = VideoFileClip("output_0002.mp4")

# Concatenate the videos sequentially
final_video = concatenate_videoclips([video1, video2, video3])
final_video.write_videofile("concatenated_video.mp4")

#Combining video and audio
video_clip = VideoFileClip("/content/drive/MyDrive/task/output/concatenated_video.mp4")
audio_clip = AudioFileClip('/content/drive/MyDrive/task/len_audio.wav')

#Saving final product as final_video.mp4
final_clip = video_clip.set_audio(audio_clip)
final_clip.write_videofile("final_video" + ".mp4", fps=60, audio_codec="aac")

/content/drive/MyDrive/taks/outt


t:   8%|▊         | 51/600 [01:39<05:51,  1.56it/s, now=None]

Moviepy - Building video concatenated_video.mp4.
Moviepy - Writing video concatenated_video.mp4




t:   8%|▊         | 51/600 [08:35<05:51,  1.56it/s, now=None]

Moviepy - Done !
Moviepy - video ready concatenated_video.mp4


...

Moviepy - Building video final1.mp4.
MoviePy - Writing audio in final1TEMP_MPY_wvf_snd.mp4



t:   8%|▊         | 51/600 [08:38<05:51,  1.56it/s, now=None]

MoviePy - Done.
Moviepy - Writing video final1.mp4




t:   8%|▊         | 51/600 [20:13<05:51,  1.56it/s, now=None]

Moviepy - Done !
Moviepy - video ready final1.mp4


In [ ]:
#Download the final output
files.download("/content/drive/MyDrive/task/output/final_video.mp4")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>